# Assignment 4

**Due to**: TBD

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Fact checking, Neural Languange Inference (**NLI**)

In [ ]:
!pip install pytorch-crf
import os
from urllib import request
import zipfile
import pandas as pd
import glob
import io
import gensim
import gensim.downloader as gloader
import numpy as np
from collections import OrderedDict
import scipy
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import f1_score
import random
import matplotlib.pyplot as plt
from torchcrf import CRF

# Intro

This assignment is centred on a particular and emerging NLP task, formally known as **fact checking** (or fake checking). As AI techniques become more and more powerful, reaching amazing results, such as image and text generation, it is more than ever necessary to build tools able to distinguish what is real from what is fake.

Here we focus on a small portion of the whole fact checking problem, which aims to determine whether a given statement (fact) conveys a trustworthy information or not. 

More precisely, given a set of evidences and a fact to verify, we would like our model to correctly predict whether the fact is true or fake.

In particular, we will see:

*   Dataset preparation (analysis and pre-processing)
*   Problem formulation: multi-input binary classification
*   Defining an evaluation method
*   Simple sentence embedding
*   Neural building blocks
*   Neural architecture extension

# The FEVER dataset

First of all, we need to choose a dataset. In this assignment we will rely on the [FEVER dataset](https://fever.ai).

The dataset is about facts taken from Wikipedia documents that have to be verified. In particular, facts could face manual modifications in order to define fake information or to give different formulations of the same concept.

The dataset consists of 185,445 claims manually verified against the introductory sections of Wikipedia pages and classified as ```Supported```, ```Refuted``` or ```NotEnoughInfo```. For the first two classes, systems and annotators need to also return the combination of sentences forming the necessary evidence supporting or refuting the claim.

## Dataset structure

Relevant data is divided into two file types. Information concerning the fact to verify, its verdict and associated supporting/opposing statements are stored in **.jsonl** format. In particular, each JSON element is a python dictionary with the following relevant fields:

*    **ID**: ID associated to the fact to verify.

*    **Verifiable**: whether the fact has been verified or not: ```VERIFIABLE``` or ```NOT VERIFIABLE```.
    
*    **Label**: the final verdict on the fact to verify: ```SUPPORTS```, ```REFUTES``` or ```NOT ENOUGH INFO```.
    
*    **Claim**: the fact to verify.
    
*    **Evidence**: a nested list of document IDs along with the sentence ID that is associated to the fact to verify. In particular, each list element is a tuple of four elements: the first two are internal annotator IDs that can be safely ignored; the third term is the document ID (called URL) and the last one is the sentence number (ID) in the pointed document to consider.

**Some Examples**

---

**Verifiable**

{"id": 202314, "verifiable": "VERIFIABLE", "label": "REFUTES", "claim": "The New Jersey Turnpike has zero shoulders.", "evidence": [[[238335, 240393, "New_Jersey_Turnpike", 15]]]}

---

**Not Verifiable**

{"id": 113501, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Grease had bad reviews.", "evidence": [[[133128, null, null, null]]]}

---

## Some simplifications and pre-processing

We are only interested in verifiable facts. Thus, we can filter out all non-verifiable claims.

Additionally, the current dataset format does not contain all necessary information for our classification purposes. In particular, we need to download Wikipedia documents and replace reported evidence IDs with the corresponding text.

Don't worry about that! We are providing you the already pre-processed dataset so that you can concentrate on the classification pipeline (pre-processing, model definition, evaluation and training).

You can download the zip file containing all set splits (train, validation and test) of the FEVER dataset by clicking on this [link](https://drive.google.com/file/d/1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1/view?usp=sharing). Alternatively, run the below code cell to automatically download it on this notebook.

**Note**: each dataset split is in .csv format. Feel free to inspect the whole dataset!

In [ ]:
import os
import requests
import zipfile

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

Download completed!
Extracting dataset...
Extraction completed!


# Classification dataset

At this point, you should have a reay-to-go dataset! Note that the dataset format changed as well! In particular, we split the evidence set associated to each claim, in order to build (claim, evidence) pairs. The classification label is propagated as well.

We'll motivate this decision in the next section!

Just for clarity, here's an example of the pre-processed dataset:

---

**Claim**: "Wentworth Miller is yet to make his screenwriting debut."

**Evidence**: "2	He made his screenwriting debut with the 2013 thriller film Stoker .	Stoker	Stoker (film)"

**Label**: Refutes

---

**Note**: The dataset requires some text cleaning as you may notice!


In [ ]:
files = ['train_pairs', 'val_pairs', 'test_pairs']
df1 = pd.DataFrame(columns=['Claim', 'Evidence', 'Label', 'split'])
for f in files:
  data = pd.read_csv ('dataset/'+f+'.csv', usecols= ['Claim','Evidence', 'Label'])
  if (f == 'train_pairs'):
   data['split'] = 'train'
  elif (f == 'val_pairs'):
    data['split'] = 'val'
  else:
    data['split'] = 'test'
  df1 = pd.concat([df1, data], axis=0)
  df = df1.rename(columns = {'Claim': 'claim', 'Evidence': 'evidence', 'Label': 'label', 'split': 'split'}, inplace = False)


# Extract training set
train_df = df.loc[df['split'] == 'train']

# Extract validation set
val_df = df.loc[df['split'] == 'val']

# Extract testing set
test_df = df.loc[df['split'] == 'test']

##GLOVE

In [ ]:
# Define function to download GloVe embedding model
def load_glove_embedding_model(embedding_dimension=50):
    """
    Loads a pre-trained GloVe word embedding model via gensim library.

    :param embedding_dimension: size of the embedding space to consider

    :return
        - pre-trained GloVe word embedding model (gensim KeyedVectors object)
    """

    download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
    embedding_model = gloader.load(download_path)
    return embedding_model

# Create embedding model
embedding_dimension = 300
embedding_model = load_glove_embedding_model(embedding_dimension)

[================================================--] 97.4% 366.5/376.1MB downloaded

##Vocabulary

In [ ]:
# Define function that builds the vocabulary
def build_vocabulary(df):
    """
    Given a dataset, builds the corresponding word vocabulary.

    :param df: dataset from which we want to build the word vocabulary (pandas.DataFrame)
    :return:
      - word vocabulary: vocabulary index to word
      - inverse word vocabulary: word to vocabulary index
      - word listing: set of unique terms that build up the vocabulary
    """
    
    word_listing = []
    for text in df['claim'].values:
        word_listing.extend(text)
    for text in df['evidence'].values:
      word_listing.extend(text)
    word_listing = list(np.unique(word_listing))

    word_indexes = range(len(word_listing))
    word_to_idx = OrderedDict(zip(word_listing, word_indexes))
    idx_to_word = OrderedDict(zip(word_indexes, word_listing))

    return idx_to_word, word_to_idx, word_listing

# Build train vocabulary
print("Building train vocabulary...")
train_idx_to_word, train_word_to_idx, train_word_listing = build_vocabulary(train_df)
print("Done!")

# Build val vocabulary
print("Building validation vocabulary...")
val_idx_to_word, val_word_to_idx, val_word_listing = build_vocabulary(val_df)
print("Done!")

# Build test vocabulary
print("Building test vocabulary...")
test_idx_to_word, test_word_to_idx, test_word_listing = build_vocabulary(test_df)
print("Done!")

Building train vocabulary...
Done!
Building validation vocabulary...
Done!
Building test vocabulary...
Done!


##Check_OOV_terms

In [ ]:
# Define function to compute OOV terms
def check_OOV_terms(embedding_model, word_listing):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_listing: dataset specific vocabulary (list)

    :return
        - list of OOV terms
    """

    oov_terms = list(filter(lambda word: word not in embedding_model, word_listing))
    return np.asarray(oov_terms)

# Extract OOV terms for the training set
print("Extracting training OOV terms...")
train_oov_terms = check_OOV_terms(embedding_model, train_word_listing)
print("Done!")

# Extract OOV terms for the validation set 
print("Extracting validation OOV terms...")
val_oov_terms = check_OOV_terms(embedding_model, val_word_listing)
print("Done!")

# Extract OOV terms for the testing set 
print("Extracting testing OOV terms...")
test_oov_terms = check_OOV_terms(embedding_model, test_word_listing)
print("Done!")

Extracting training OOV terms...
Done!
Extracting validation OOV terms...
Done!
Extracting testing OOV terms...
Done!


##co_occurrence_count

In [ ]:
# Define co-occurrence matrix build function
def co_occurrence_count(df, idx_to_word, word_to_idx, window_size=1):
    """
    Builds word-word co-occurrence matrix based on word counts.

    :param df: pre-processed dataset (pandas.DataFrame)
    :param idx_to_word: vocabulary map (index -> word) (dict)
    :param word_to_idx: vocabulary map (word -> index) (dict)

    :return
      - co_occurrence symmetric matrix of size |V| x |V| (|V| = vocabulary size)
    """

    data = []
    row = []
    col = []

    claim_corpus = df['claim'].tolist()
    evidence_corpus = df['evidence'].tolist()
    corpus = claim_corpus + evidence_corpus
    for text in corpus:
        for pos, word in enumerate(text):
            i = word_to_idx.get(word)
            start = max(0, pos - window_size)
            end = min(len(text), pos + window_size + 1)
            for pos2 in range(start, end):
                if text[pos2] != word: 
                    j = word_to_idx.get(text[pos2])
                    data.append(1.)
                    row.append(i)
                    col.append(j)
    
    co_occurrence = scipy.sparse.csr_matrix((data,(row,col)))
    return co_occurrence

# Build train co-occurrence matrix
print("Build train co-occurrence matrix... (it may take a while...)")
train_co_occurrence_matrix = co_occurrence_count(train_df, train_idx_to_word, train_word_to_idx)
print("Done!")

# Build validation co-occurrence matrix
print("Build validation co-occurrence matrix... (it may take a while...)")
val_co_occurrence_matrix = co_occurrence_count(val_df, val_idx_to_word, val_word_to_idx)
print("Done!")

# Build test co-occurrence matrix
print("Build test co-occurrence matrix... (it may take a while...)")
test_co_occurrence_matrix = co_occurrence_count(test_df, test_idx_to_word, test_word_to_idx)
print("Done!")

Build train co-occurrence matrix... (it may take a while...)
Done!
Build validation co-occurrence matrix... (it may take a while...)
Done!
Build test co-occurrence matrix... (it may take a while...)
Done!


##build_embedding_matrix

In [ ]:
# Define function to build embedding matrix
def build_embedding_matrix(embedding_model, embedding_dimension, word_to_idx, idx_to_word, oov_terms, co_occurrence_count_matrix, random):
    """
    Builds the embedding matrix of a specific dataset given a pre-trained word embedding model

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param embedding_dimension: dimension of the embedding model
    :param word_to_idx: vocabulary map (word -> index) (dict)
    :param word_to_idx: vocabulary map (index -> word) (dict)
    :param oov_terms: list of OOV terms (list)
    :param co_occorruence_count_matrix: the co-occurrence count matrix of the given dataset (window size 1)
    :param random: boolean value to define how to manage oov terms, if true use random, otherwise use neighbors 

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the dataset specific vocabulary (shape |V| x d)
    """

    word_listing = list(word_to_idx.keys())
    V = len(word_listing)

    max, min = -np.inf, np.inf
    for word in word_listing:
        if word not in oov_terms:
            word_embedding = embedding_model[word]
            curr_max = np.max(word_embedding)
            curr_min = np.min(word_embedding)
            if curr_max > max:
                max = curr_max
            if curr_min < min:
                min = curr_min

    embedding_matrix = np.zeros(shape=(V, embedding_dimension))
    for word, index in tqdm(word_to_idx.items()):
        if word in oov_terms:
            if random:
                embedding_matrix[index] = np.random.uniform(low=min, high=max, size=(embedding_dimension,))
            else:
                neighbors = co_occurrence_count_matrix[index].nonzero()[1]
                neighbors_words = np.array([idx_to_word.get(neighbor) for neighbor in neighbors])
                non_oov_neighbors = np.setdiff1d(neighbors_words, oov_terms)
                if len(non_oov_neighbors) == 0:
                    embedding_matrix[index] = np.random.uniform(low=min, high=max, size=(embedding_dimension,))
                else:
                    non_oov_neighbors_weights = np.array([])
                    for non_oov_word in non_oov_neighbors:
                        non_oov_index = word_to_idx.get(non_oov_word)
                        non_oov_neighbors_weights = np.append(non_oov_neighbors_weights, co_occurrence_count_matrix[index, non_oov_index])
                    non_oov_neighbors_embeddings = embedding_model[non_oov_neighbors]
                    embedding_matrix[index] = np.average(non_oov_neighbors_embeddings, weights=non_oov_neighbors_weights, axis=0)
        else:
            embedding_matrix[index] = embedding_model[word]

    return embedding_matrix

random_oov = False

# Build train embedding matrix
train_embedding_matrix = build_embedding_matrix(embedding_model, embedding_dimension, train_word_to_idx, train_idx_to_word, train_oov_terms, train_co_occurrence_matrix, random_oov)

# Build val embedding matrix
val_embedding_matrix = build_embedding_matrix(embedding_model, embedding_dimension, val_word_to_idx, val_idx_to_word, val_oov_terms, val_co_occurrence_matrix, random_oov)

# Build val embedding matrix
test_embedding_matrix = build_embedding_matrix(embedding_model, embedding_dimension, test_word_to_idx, test_idx_to_word, test_oov_terms, test_co_occurrence_matrix, random_oov)

##build_label_dic

In [ ]:
# Define a function to build the indexing of the POS tags
def build_label_dic(df):
    """
      Given a dataset, builds the corresponding Label indexing.

      :param df: dataset from which we want to build the Label indexing (pandas.DataFrame)
      
      :return:
        - Label index: vocabulary label to index
    """

    label_listing = []
    label_listing.extend(df['label'])
    label_listing = list(np.unique(label_listing))

    label_indexes = range(0, len(label_listing)+1)
    label_to_idx = OrderedDict(zip(label_listing, label_indexes))

    return label_to_idx

# Index of the POS tags
print("Building dictionary for label...")
label_to_idx = build_label_dic(df)
print(label_to_idx)

Building dictionary for label...
OrderedDict([('REFUTES', 0), ('SUPPORTS', 1)])


## prepare_sequences

In [ ]:
# Define a function to prepare some sequences for the training, validation and testing phases
def prepare_sequences(df, word_to_idx, embedding_matrix, tag_to_idx):
    x1 = []
    x2 = []
    y = []
    x1_lengths = []
    x2_lengths = []
    for _, row in df.iterrows():
        text1, text2, labels = row['claim'], row['evidence'], row['label']

        text1_length = len(text1)
        text2_length = len(text2)
        
        x1_lengths.append(text1_length)
        x2_lengths.append(text2_length)
        
        token1_embeddings = [embedding_matrix[word_to_idx[token]] for token in text1]
        token2_embeddings = [embedding_matrix[word_to_idx[token]] for token in text2]

        token1_embeddings = np.array(token1_embeddings, dtype=np.float32)
        token2_embeddings = np.array(token2_embeddings, dtype=np.float32)

        token1_embeddings = torch.from_numpy(token1_embeddings)
        token2_embeddings = torch.from_numpy(token2_embeddings)

        x1.append(token1_embeddings)
        x2.append(token2_embeddings)

        label_encodings = [label_to_idx[tag] for label in labels]
        label_encodings = np.array(label_encodings)
        label_encodings = torch.from_numpy(label_encodings)
        y.append(label_encodings)
    
    return x1, x2, y, x1_lengths, x2_lengths

# Problem formulation

As mentioned at the beginning of the assignment, we are going to formulate the fact checking problem as a binary classification task.

In particular, each dataset sample is comprised of:

*     A claim to verify
*     A set of semantically related statements (evidence set)
*     Fact checking label: either evidences support or refute the claim.

Handling the evidence set from the point of view of neural models may imply some additional complexity: if the evidence set is comprised of several sentences we might incur in memory problems.

To this end, we further simplify the problem by building (claim, evidence) pairs. The fact checking label is propagated as well.

Example:

     Claim: c1 
     Evidence set: [e1, e2, e3]
     Label: S (support)

--->

    (c1, e1, S),
    (c1, e2, S),
    (c1, e3, S)

## Schema

The overall binary classification problem is summed up by the following (simplified) schema

![](https://drive.google.com/uc?export=view&id=1Wm_YBnFwgJtxcWEBpPbTBEVkpKaL08Jp)

Don't worry too much about the **Encoding** block for now. We'll give you some simple guidelines about its definition. For the moment, stick to the binary classification task definition where, in this case, we have 2 inputs: the claim to verify and one of its associated evidences.

# Architecture Guidelines

There are many neural architectures that follow the above schema. To avoid phenomena like the writer's block, in this section we are going to give you some implementation guidelines.

In particular, we would like you to test some implementations so that you explore basic approaches (neural baselines) and use them as building blocks for possible extensions.

## Handling multiple inputs

The first thing to notice is that we are in a multi-input scenario. In particular, each sample is comprised of a fact and its asssociated evidence statement.

Each of these input is encoded as a sequence of tokens. In particular, we will have the following input matrices:

*    Claim: [batch_size, max_tokens]
*    Evidence: [batch_size, max_tokens]

Moreover, after the embedding layer, we'll have:

*    Claim: [batch_size, max_tokens, embedding_dim]
*    Evidence: [batch_size, max_tokens, embedding_dim]

But, we would like to have a 2D input to our classifier, since we have to give an answer at pair level. Therefore, for each sample, we would expect the following input shape to our classification block:

*   Classification input shape: [batch_size, dim]

**How to do that?**

We inherently need to reduce the token sequence to a single representation. This operation is formally known as **sentence embedding**. Indeed, we are trying to compress the information of a whole sequence into a single embedding vector.

Here are some simple solutions that we ask you to try out:

*   Encode token sequences via a RNN and take the last state as the sentence embedding.

*   Encode token sequences via a RNN and average all the output states.

*   Encode token sequences via a simple MLP layer. In particular, if your input is a [batch_size, max_tokens, embedding_dim] tensor, the matrix multiplication works on the **max_tokens** dimension, resulting in a [batch_size, embedding_dim] 2D matrix.

*   Compute the sentence embedding as the mean of its token embeddings (**bag of vectors**).

## Merging multi-inputs

At this point, we have to think about **how** we should merge evidence and claim sentence embeddings.

For simplicity, we stick to simple merging strategies:

*     **Concatenation**: define the classification input as the concatenation of evidence and claim sentence embeddings

*     **Sum**: define the classification input as the sum of evidence and claim sentence embeddings

*     **Mean**: define the classification input as the mean of evidence and claim sentence embeddings

For clarity, if we the sentence embedding of a single input has shape [batch_size, embedding_dim], then the classification input has shape:

*     **Concatenation**: [batch_size, 2 * embedding_dim]

*     **Sum**: [batch_size, embedding_dim]

*     **Mean**: [batch_size, embedding_dim]

# A simple extension

Lastly, we ask you to modify previously defined neural architectures by adding an additional feature to the classification input.

We would like to see if some similarity information between the claim to verify and one of its associated evidence might be useful to the classification.

Compute the cosine similarity metric between the two sentence embeddings and concatenate the result to the classification input.

For clarity, since the cosine similarity of two vectors outputs a scalar value, the classification input shape is modified as follows:

*     **Concatenation**: [batch_size, 2 * embedding_dim + 1]

*     **Sum**: [batch_size, embedding_dim + 1]

*     **Mean**: [batch_size, embedding_dim + 1]



# Performance evaluation

Due to our simplifications, obtained results are not directly compatible with a traditional fact checking method that considers the evidence set as a whole.

Thus, we need to consider two types of evaluations.

**Multi-input classification evaluation**

This type of evaluation is the easiest and concerns computing evaluation metrics, such as accuracy, f1-score, recall and precision, of our pre-processed dataset.

In other words, we assess the performance of chosen classifiers.

**Claim verification evaluation**

However, if we want to give an answer concerning the claim itself, we need to consider the whole evidence set. 

Intuitively, for a given claim, we consider all its corresponding (claim, evidence) pairs and their corresponding classification outputs. 

At this point, all we need to do is to compute the final predicted claim label via majority voting.

Example:

    Claim: c1
    Evidence set: e1, e2, e3
    True label: S

    Pair outputs:
    (c1, e1) -> S (supports)
    (c1, e2) -> S (supports)
    (c1, e3) -> R (refutes)

    Majority voting:
    S -> 2 votes
    R -> 1 vote

    Final label:
    c1 -> S

Lastly, we have to compute classification metrics just like before.

# Tips and Extras

## Extensions are welcome!

Is this task too easy for you? Are you curious to try out things you have seen during lectures (e.g. attention)? Feel free to try everything you want!

Don't forget to try neural baselines first!

## Comments and documentation

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

## Organization

We suggest you to divide your work into sections. This allows you to build clean and modular code, as well as easy to read and to debug.

A possible schema:

*   Dataset pre-processing
*   Dataset conversion
*   Model definition
*   Training
*   Evaluation
*   Comments/Summary

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

Don't forget that your feedback is very important! Your suggestions help us improving course material.

# FAQ

---

**Q: Can I do something text pre-processing?**

**A:** You have to! If you check text data, the majority of sentences need some cleaning.

---

**Q: I'm struggling with the implementation. Can you help me?**

**A:** Yes sure! Write us an email about your issue. If you are looking for a particular type of operation, you can easily check the documentation of the deep learning framework you are using (google is your friend).

---

**Q: Can I try other encoding strategies or neural architectures?**

**A:** Absolutely! Remember to try out recommended neural baselines first and only then proceed with your extensions.

---